### Mouse Slide-seq Prep

This notebooks shows how Slide-seq data is pre-processed to run slideCNA. Data comes from a healthy mouse Slide-seq sample: Stickels, Murray, et al. Nat. Biotech 2020, https://www.nature.com/articles/s41587-020-0739-1.

The count data can be downloaded from https://singlecell.broadinstitute.org/single_cell/study/SCP815/highly-sensitive-spatial-transcriptomics-at-near-cellular-resolution-with-slide-seqv2#study-download (Puck_200115_08). Since the portal requires an account, no scripted download is possible. To generate the anndata object and metadata file the following files are required:
- Puck_200115_08_bead_locations.csv
- Puck_200115_08.digital_expression.txt

In [1]:
import scanpy as sc
import pandas as pd
import polars as pl
import numpy as np
import tacco as tc

In [ ]:
path_to_data <- "path/to/data/"
path_to_results <- "path/to/results/

In [3]:
adata = sc.read_text(path_to_data + 'Puck_200115_08.digital_expression.txt', first_column_names=False)
adata = adata.T
adata

AnnData object with n_obs × n_vars = 53208 × 23264

In [4]:
# Filter and subsample (optional)
adata = tc.pp.filter(adata, min_genes_per_cell=700, min_cells_per_gene=15, return_view=False)
# sc.pp.subsample(adata, fraction=0.4) #remove for full-run
adata

AnnData object with n_obs × n_vars = 10114 × 15856

In [5]:
adata.var["mt"] = adata.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)

In [6]:
# Read spatial information
df = pd.read_csv(path_to_data + 'Puck_200115_08_bead_locations.csv', sep='\s+|,', engine='python')
df.set_index('barcodes', drop=False, inplace=True)
df.rename(columns={'xcoord':'pos_x', 'ycoord':'pos_y'}, inplace=True)

In [7]:
adata.obs['pos_x']=df['pos_x']
adata.obs['pos_y']=df['pos_y']

adata.obsm['counts'] = adata.X # Helper function expects raw counts in .obsm
adata.uns['counts_var'] = adata.var.index.to_numpy() # Helper function expects gene names under .uns
adata.obs['nCount_RNA'] = adata.X.sum(axis=1)

# Set 25% of the data as non-malignant reference
adata.obs['cluster_type'] = 'Malignant'
reference_index = adata.obs.sample(frac=0.25).index
adata.obs.loc[reference_index, 'cluster_type'] = 'Normal'

adata.obs['bc'] = adata.obs_names

In [8]:
# Write-out metadata (beads_df) and adata
adata.obs.to_csv(path_to_results + 'beads_df.csv',)
adata.write(path_to_results + 'Puck.h5ad')
adata

AnnData object with n_obs × n_vars = 10114 × 15856
    obs: 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'pos_x', 'pos_y', 'nCount_RNA', 'cluster_type', 'bc'
    var: 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'counts_var'
    obsm: 'counts'

Next, prepare genomic positions file. The base file can be downloaded here: https://data.broadinstitute.org/Trinity/CTAT/cnv/ where we need the *mouse_gencode.GRCm39.vM32.basic.annotation.by_gene_name.infercnv_positions* file (mouse). Further information regarding this file can be found under https://github.com/broadinstitute/inferCNV/wiki/File-Definitions. 

In [17]:
df = pl.read_csv(path_to_data + 'mouse_gencode.GRCm39.vM32.basic.annotation.by_gene_name.infercnv_positions.csv', has_header=False)
df

column_1,column_2,column_3,column_4
str,str,i64,i64
"""4933401J01Rik""","""chr1""",3143476,3144545
"""Gm26206""","""chr1""",3172239,3172348
"""Xkr4""","""chr1""",3276124,3741721
"""Gm18956""","""chr1""",3322980,3323459
"""Gm37180""","""chr1""",3435954,3438772
…,…,…,…
"""mt-Nd6""","""chrM""",13552,14070
"""mt-Te""","""chrM""",14071,14139
"""mt-Cytb""","""chrM""",14145,15288


In [18]:
df = df.rename({'column_1':'GENE','column_2':'chr','column_3':'start','column_4':'end'})
df = df.with_columns(
   pl.col('chr').cum_count().over(['chr']).alias("rel_gene_pos")
)
df

GENE,chr,start,end,rel_gene_pos
str,str,i64,i64,u32
"""4933401J01Rik""","""chr1""",3143476,3144545,1
"""Gm26206""","""chr1""",3172239,3172348,2
"""Xkr4""","""chr1""",3276124,3741721,3
"""Gm18956""","""chr1""",3322980,3323459,4
"""Gm37180""","""chr1""",3435954,3438772,5
…,…,…,…,…
"""mt-Nd6""","""chrM""",13552,14070,33
"""mt-Te""","""chrM""",14071,14139,34
"""mt-Cytb""","""chrM""",14145,15288,35


In [19]:
df.write_csv(path_to_results + 'mouse_gencode.GRCm39.vM32.basic.annotation.by_gene_name.infercnv_positions_rel_gene_pos.csv', )